In [3]:
# imports
from bs4 import BeautifulSoup as BS
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [4]:
# set plotting params
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 18
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 18
plt.rcParams['figure.titlesize'] = 24

In [5]:
file_path = '../data/ufodata.json'

In [35]:
reports = []
with open(file_path) as f:
    for i in f:
        reports.append(json.loads(i))
            
print(len(reports))

136494


In [9]:
class CleanUFOs:
    def __init__(self, file_path):
        self.file_path = file_path
        self.rows = []
        self.load_data()
        
    def load_data(self):
        self.reports = []
        with open(self.file_path) as f:
            for i in f:
                self.reports.append(json.loads(i))
                
    def parse_html(self, report):
        try:
            bs = BS(report['html'], 'html.parser')

            text = str(bs.find('tbody'))
            text_lst = text.split(' : ')
            occured = text_lst[1]
            occured = occured.split('  ')[0].strip()

            body = text_lst[2]
            body_lst = body.split(': ')

            reported = body_lst[1]
            reported = reported.split('<')[0].strip()

            location = body_lst[3]
            location = location.split('<')[0]

            city, state = location.split(', ')

            shape = body_lst[4].split('<')[0].strip()

            body = body_lst[4].split(':')

            duration = body[1]
            duration = duration.split('<')[0]

            description = text_lst[-1].split('">')[3].split('</font>')[0].replace('<br/>', '').strip()

            data_dct = {'occured': occured, 'reported': reported, 'city': city, 'state': state,
                        'shape': shape, 'duration': duration, 'description': description}

            self.rows.append(data_dct)
        except:
            pass
    
    def clean_reports(self):
        for i, report in enumerate(self.reports):
            self.parse_html(report)
            if i % 1000 == 0:
                print(i)
    
    def to_pandas(self):
        return pd.DataFrame(self.rows)

In [10]:
clean_ufos = CleanUFOs(file_path)

In [11]:
clean_ufos.clean_reports()
df = clean_ufos.to_pandas()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000


In [12]:
df

,occured,reported,city,state,shape,duration,description
0,5/6/2017 05:00,5/6/2017 4:10:01 AM 04:10,Camp McGregor,NM,Light,10 minutes,Light seen over mountain's east of Camp McGreg...
1,5/6/2017 04:50,5/6/2017 5:00:54 AM 05:00,Mojave (Canada),BC,Light,1,Light in sky stationary. Not a airplane or an...
2,5/5/2017 11:30,5/5/2017 12:18:44 PM 12:18,Austin,TX,Disk,3 seconds,"Flying saucer descends, possibly lands in Nort..."
3,5/5/2017 03:00,5/5/2017 3:49:05 AM 03:49,El Mirage,AZ,Circle,30 seconds,"While letting my dog out, a very bright white ..."
4,5/4/2017 23:34,5/4/2017 10:38:52 PM 22:38,York,NE,Fireball,0,A fire ball was moving in the atmosphere while...
...,...,...,...,...,...,...,...
131980,5/15/1982 20:00,12/8/1996 01:45,West Reno (out on old Hwy 40),NV,Other,1 minute,"SUMMARY: Loud boom, so loud shook house. Wen..."
131981,8/15/1979 22:10,12/17/1996 13:50,Beach Haven Terrace (Long Beach Island),NJ,Fireball,4-5 seconds,SUMMARY: Fireball flying parellel with the ho...
131982,7/1/1972 19:00,12/10/1996 19:21,Provo (south of),UT,Disk,20-30 min.,"SUMMARY: 1972, Utah-small bright cloud, I'm d..."
131983,7/15/1965 16:00,1/28/1997 17:27,Nyona Lake,IN,Cigar,30 minutes,"SUMMARY: Cigar shaped object. 25,000 to 40,00..."


In [13]:
df.shape

(131985, 7)